https://medium.com/@great.investor/how-to-scrape-data-from-yahoo-finance-with-python-and-selenium-81e8577db69c


In [28]:
# import webdriver
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service

In [29]:
# Add Tags here
taglist =["MU", "MSFT"]

In [30]:
# ✅ Initialize the DataFrame BEFORE any function uses it
StatisticsSummary = pd.DataFrame()

In [31]:
# ✅ Use a fixed msedgedriver.exe path
service = Service(r"C:\Windows\System32\msedgedriver.exe")
driver = webdriver.Edge(service=service)

In [ ]:
# get yahoo finance.com
is_link = 'https://finance.yahoo.com/lookup'
driver.get(is_link)
time.sleep(5)

In [ ]:
# Search for an elements of taglist to disable the button that appears after first search
def SearchElements(tag):
    SearchBar = driver.find_element(By.XPATH, "//form[@class='_yb_1vht69d _yb_1ndj1ez ybar-enable-search-ui ybar-assist_billboard_v2 _yb_oow1q0 _yb_9vl95e _yb_1o792pg']//input[@class='_yb_d7nywr _yb_owcm9n _yb_ikgg1k finsrch-inpt ']")
    SearchBar.send_keys(tag)
    SearchBar.send_keys(Keys.ENTER)
    time.sleep(5)

In [ ]:
def getStatistics(tag):
    global StatisticsSummary
    # statistic button을 찾음
    StatisticsBtn = driver.find_element(By.XPATH, "//li[@class='yf-1mwz2jq']/a[@category='key-statistics']")
    action = ActionChains(driver)
    action.click(on_element=StatisticsBtn)
    action.perform()
    time.sleep(5)

    # VAluation Measures 테이블 첫번째 행 레이블(thead) 추출
    labelElements = driver.find_elements(
    By.XPATH, "//div[@class='table-container yf-kbx2lo']//table[@class='table yf-kbx2lo']//thead//tr[@class='yf-kbx2lo']//th[@class='yf-kbx2lo']")

    # 각 element에서 text 추출
    labelTexts = [el.text for el in labelElements if el.text.strip() != '']

    # VAluation Measures 테이블 2번째행과 이후에 있는 데이터(tbody) 추출
    bodyElements = driver.find_elements(
        By.XPATH, "//div[@class='table-container yf-kbx2lo']//table[@class='table yf-kbx2lo']//tbody//tr[@class='yf-kbx2lo alt' or @class='yf-kbx2lo']//td[@class='yf-kbx2lo']")

    dataList = [value.text for value in bodyElements]

    # time label을 저장하기 위해 dictionary 생성
    time_values = labelTexts[:] 

    # statistics를 저장하기 위해 dictionary 생성
    column_labels = dataList[0::7]

    # Extract the dataList rows (all elements except the labels, reshaped into columns)
    data_values = [dataList[i::7] for i in range(1, 7)]

    # tag label을 위해 dataframe생성
    df = pd.DataFrame(data=data_values, columns= column_labels)   
    # tag와 time 컬럼을 df에 직접 삽입
    df.insert(0, 'tag', tag)            # add tag as second column
    df.insert(1, 'time', time_values)   # add as first column
    # combained dataframe에 StatisticsSummary에 append
    StatisticsSummary = pd.concat([StatisticsSummary, df], ignore_index=True)


In [35]:
for tag in taglist:
    SearchElements(tag)
    getStatistics(tag)

ValueError: Length of values (0) does not match length of index (6)

In [ ]:
# output을 확인하기 위해 프린트
print(StatisticsSummary)
print(f"✅ Finished {tag} ({len(StatisticsSummary)} rows total)")

     tag        time Market Cap Enterprise Value Trailing P/E Forward P/E  \
0     MU     Current    205.64B          210.61B        24.21       12.33   
1     MU   8/31/2025    133.19B          138.52B        21.44        9.07   
2     MU   5/31/2025    105.70B          112.44B        22.60        9.03   
3     MU   2/28/2025    104.68B          111.46B        26.83       12.76   
4     MU  11/30/2024    109.12B          115.02B       139.93       11.03   
5     MU   8/31/2024    106.71B          112.20B           --       11.17   
6   MSFT     Current      3.83T            3.80T        37.81       33.33   
7   MSFT   6/30/2025      3.70T            3.68T        38.44       33.11   
8   MSFT   3/31/2025      2.79T            2.78T        30.25       25.06   
9   MSFT  12/31/2024      3.13T            3.12T        34.81       32.05   
10  MSFT   9/30/2024      3.20T            3.19T        36.47       32.57   
11  MSFT   6/30/2024      3.32T            3.32T        38.73       33.33   

In [ ]:
# use os.path.expanduser
output_path = os.path.expanduser("~/Dropbox/python-finance-lecture/yahoo-finance-scraping/youtube-scraping/results/StatisticsSummary.xlsx")
StatisticsSummary.to_excel(output_path,sheet_name='statistics_summary')


NameError: name 'os' is not defined

In [ ]:
# add a polite driver close
driver.quit()